## 产生聚类

In [18]:
import gensim
import pandas as pd
import random
import csv
from sklearn.cluster import KMeans
from sklearn.externals import joblib
import pickle

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


问题1：playover50是什么？  
videoId,playNum  
t0153fxex5j,187  
k00153ct6t7,1698  
y0173rhl44f,54  
u0020kmb1s8,648  
m0121n2s61s,60  
t001019SiAU,9641

问题2：为什么要随机抽取50000个

In [120]:
class gen_clustering_model_():
    def __init__(self, n, path_train):
        self.path_train = path_train
        self.n = n
        
    def gen_popularity(self):
        f=open(self.path_train, 'rb')
        train = f.read()
        f.close()
        train = eval(train)
        video_popularity = {}
        for i in train.keys():
            for j in train[i].strip('\n').strip(' ').split(' '):
                if j.strip('') not in video_popularity:
                    video_popularity[j]=1
                else:
                    video_popularity[j] += 1
        self.video = pd.DataFrame([video_popularity.keys(), video_popularity.values()]).T
        self.video.columns = ['userId', 'playNum']
        self.video.to_csv('D:/wenjun/xiaoming/video_popularity.csv', index=False, sep=',')
        self.playOver50 = self.video[self.video['playNum']>50]
        self.playOver50.to_csv('D:/wenjun/xiaoming/playOver50.csv', index=False, sep=',')
        
    def geneVideoVec(self):
        data = pd.read_csv('D:/wenjun/xiaoming/playOver50.csv')
        videox = data.videoId
        videos = random.sample(list(videox), 50000)
        self.model = gensim.models.Word2Vec.load('D:/wenjun/word2vec9day/word2vec.model')
        dicc = {}
        count = 0
        for video in videos:
            # print video
            if count % 1000 == 0:
                print (count)
            count += 1
            try:
                dicc[video.strip('\n')] = self.model[video.strip('\n')]
            except:
                continue
       #第一个改动，删除以下四句代码，因为在python3里面不能把dicc.values()写入csv中，另外再把dicc写入video_vec_dictionary中。         
#        f = open('D:/wenjun/xiaoming/videoVec.csv', 'wb')
#        writer = csv.writer(f)
#        writer.writerows(dicc.values())
#        f.close()
        pd.DataFrame(list(dicc.values())).to_csv('D:/wenjun/xiaoming/videoVec.csv', index=False, header=False)
        with open('D:/wenjun/xiaoming/video_vec_dictionary.pkl', 'wb') as f:
            pickle.dump(dicc, f, pickle.HIGHEST_PROTOCOL)
            
            
    def clustering_model(self):
        rawData = open('D:/wenjun/xiaoming/videoVec.csv').readlines()
        rawVec = []
        for i in rawData:
            kk = i.split(',')
            rawVec.append(kk)
        kmeans = KMeans(n_clusters=self.n).fit(rawVec)
        target = kmeans.labels_
        joblib.dump(kmeans, 'D:/wenjun/xiaoming/clustering_model_C_'+str(self.n))

In [121]:
#假设产生的聚类中心数为60
gen_clustering_model = gen_clustering_model_(60, 'D:/Elias/50000/pos.txt')

In [122]:
gen_clustering_model.gen_popularity()

In [4]:
gen_clustering_model.geneVideoVec()
gen_clustering_model.clustering_model()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [5]:
word2vec_model = gen_clustering_model.model

## 计算用户对聚类中心的兴趣度

In [6]:
import numpy as np

问题3:user_history_play_for_gaussion 应该是用来预测的用户的历史浏览记录,他如何产生？
user_history_play_for_gaussion如下所示：
425480c23d731034a4d780fb5201850a	l0304cqw8ky l00201eosyc s0306f6hs31 x0195a3rnr3 v0196jcw04r v03002xjrp2 y00207o620r m0307na51tx  
93a9b622e96d1033a4d780fbd48ad00a	l0013je0mhx i03037kti8v p0304pege1a q0303sbzsth x0197ditcpq p0303uk8aux d0303j8jp9k v0192nv2y0q p0304scglh1 u01803bi5b8 g0181z9ccq1 n03034wdtbr h01696lgh6r v0303dkskrn b0302q6k8pi n0303o68gq0 i0165x5foee r0301ert7at x0300z8z7kc 

In [171]:
class gen_gaussian_():
    def __init__(self, train_path, test_path):
        self.train_path = train_path
        self.test_path = test_path
        
    def gen_user_history_play_for_gaussion(self):
        
        f=open(self.train_path, 'rb')
        train = f.read()
        f.close()
        train = eval(train)
        
        f=open(self.test_path, 'rb')
        test = f.read()
        f.close()
        test = eval(test)  
        
        user_history_play_for_gaussion = {}
        with open('D:/wenjun/xiaoming/user_history_play_for_gaussion.txt', 'wb') as f:
            for i in list(test.keys()):
                if i in list(train.keys()):
                    f.write(bytes(i, encoding = "utf8"))
                    f.write(bytes('\t', encoding = "utf8"))
                    user_history_play_for_gaussion[i] = train[i]
                    f.write(bytes(user_history_play_for_gaussion[i], encoding='utf8'))
                    f.write(bytes('\n', encoding='utf8'))
        
        
    def gaussian(self, no, sigma, cluModel, dict):
        # cmodel = joblib.load('clustering_model/clustering_model_C_'+no)
        # wmodel = gensim.models.Word2Vec.load('../code/models/outp1')
        cmodel = cluModel
        # wmodel = videoModel
        centers = cmodel.cluster_centers_
        # print len(centers[1])
        with open('D:/wenjun/xiaoming/user_history_play_for_gaussion.txt') as f:
            play_data = f.readlines()
        interest_dic = {}
        for i in play_data[90900:110900]:
            user_id, play = i.split('\t')
            play_list = play.strip().split(' ')
            inter_list = {}
            for x in range(len(centers)):
                aver_interest = []
                for video in play_list:
                    try:
                        videovec = dict[video]
                        aver_interest.append(np.e**(-(np.linalg.norm(np.array(videovec)-np.array(centers[x]))**2)/sigma))
                    except:
                        continue
                inter_list[x] = np.array(aver_interest).mean()
            interest_dic[user_id] = inter_list
            # break
        a = pd.DataFrame(interest_dic).T
        a.to_csv('D:/wenjun/xiaoming/C60/user_inter_Guassian_sigma_'+str(sigma), header=True)

In [172]:
gen_gaussian = gen_gaussian_('D:/Elias/50000/pos.txt', 'D:/Elias/50000/test_seq.txt')

In [173]:
gen_gaussian.gen_user_history_play_for_gaussion()

In [55]:
#载入聚类模型
cmodel = joblib.load('D:/wenjun/xiaoming/clustering_model_C_' + str(60))

In [56]:
#选择聚类簇数量为60，sigma为70
with open('D:/wenjun/xiaoming/video_vec_dictionary.pkl', 'rb') as f:
    dict = pickle.load(f)
gen_gaussian.gaussian(60, 70, cmodel, dict)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice.
C:\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                                        0         1         2         3   \
788891f237641034a4d780fbd48ad00a  0.062462  0.097852  0.096983  0.119551   
78bb092970aa1034a4d780fb9186d00a  0.023414  0.034890  0.024664  0.025751   
fabca6b700251034a4d780fb9286d00a  0.016486  0.021425  0.022390  0.025432   
94d7b424539c1034a4d780fb9186d00a  0.036731  0.063045  0.067770  0.082886   
ac60c670bf641033a4d780fb9286d00a  0.004474  0.008107  0.004939  0.005683   

                                        4         5         6         7   \
788891f237641034a4d780fbd48ad00a  0.098850  0.105736  0.102344  0.104229   
78bb092970aa1034a4d780fb9186d00a  0.025966  0.029675  0.028734  0.025501   
fabca6b700251034a4d780fb9286d00a  0.021205  0.026938  0.023247  0.025607   
94d7b424539c1034a4d780fb9186d00a  0.067557  0.067876  0.067872  0.068552   
ac60c670bf641033a4d780fb9286d00a  0.005055  0.009040  0.005700  0.004822   

                                        8         9     ...           50  \
788891f237

## 对候选的video求聚类的类别

In [10]:
import pandas as pd
from sklearn.externals import joblib
import gensim

问题4：候选的videoFrequency是如何产生的？
videoFrequency如下所示：  
videoId	playNum  
g0159qivjn5	2  
u01792e03a3	3  
s0302szebir	1  
y0302vtxhbc	2  
s01812m8doo	1  
t0153fxex5j	187

In [11]:
class gen_clustering_():
    
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        
    
    def getVideoList(self):
        data = pd.read_table('D:/wenjun/xiaoming/video_popularity.csv')
        video = data.videoId
        return list(video)

    def geneLabel(self):
        self.clustering_model = joblib.load('D:/wenjun/xiaoming/clustering_model_C_60')
        videos = self.getVideoList()
        video_label = []
        valid_videos = []
        count = 0
        for i in range(0, 1056):
            count += 1
            sub_videos = videos[i*10000:(i+1)*10000]
            sub_videos_vec = []
            for video in sub_videos:
                try:
                    videoVec = self.word2vec_model[video]
                except:
                    continue
                sub_videos_vec.append(list(videoVec))
                valid_videos.append(video)
            video_label.extend(self.clustering_model.predict(sub_videos_vec))
            # break
        dicc = {}
        dicc["label"] = video_label
        dicc["videoId"] = valid_videos
        pd.DataFrame(dicc).to_csv('D:/wenjun/xiaoming/video_label_C60.csv', index=False)

In [12]:
gen_clustering = gen_clustering_(word2vec_model)

In [13]:
gen_clustering.geneLabel()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

...

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__g

## 产生推荐的列表

In [14]:
import gzip

问题5:如何产生testData？   
testData如下所示：
0001b95a42061034a4d780fb0722850a	o1731uj2ljv:0.14 c1728u4nwqt:0.75 k0308r50906:0.12 w1730l3rpvt:0.25 v17282ytr4s:0.25 i1728ofph9q:0.25 d17189clmaz:0.12 r162174oe5t:0.12 a17255avwi5:0.38 k17115jfln7:0.50 o0020drtn6c:0.86 c1708g7ta65:0.50 j1627jv6au5:0.12 e1706icgcvj:0.25 e1730rh75a1:0.25 x17296t5b2p:0.38 b1720ssx951:0.88 r1731zzcaao:0.71 g17311bvf6s:0.25 l1627p9m795:0.12 d0308n59bjs:0.99 y16236cenpy:0.75 y0020kjuqzb:1.00 r162174oe5t:0.38 w1704tufk92:0.12 m173047176t:0.12 y1703l5cn2l:1.00 b1725nxgs8k:0.12 a1705wxleu7:0.38 d0020r6jjqp:0.43 m1703d62pjq:0.50 u1709m34tn5:0.12 h1720nhp9on:0.25 w1731vnc9lc:0.25 k1718pyause:0.12 e1725dxv1hz:0.38 u1709m34tn5:1.00 y17313tu6xr:0.50  
000228c12b791034a4d780fb0722850a	q03074fxbp4:0.33 r00202vjk8i:0.23 s030777udgc:0.80 z0306d1dafg:0.99 u01941qdgix:0.22 i0178sognss:0.98 w01993qzi89:0.33 i0183wklnwl:0.96 m0194krwwo3:0.82 x0193a9wyqm:0.97 e0199d28f3f:0.12 q0307nnfswc:0.71 c0306ikrfvv:0.95 v01793mprex:0.51 c0181shxe4h:0.30 e0306ooq3db:0.41 n01840heuuw:0.47 f0307rvbh6s:0.45 y0179zdluq7:0.88 k03066fhnol:0.99 b0189w6h595:0.58 d0307z37hdz:1.00 n01409cgue6:0.27 w0305x735j8:0.16 b01784xj94m:0.99


In [67]:
def delSame(aList, bList):
    for i in bList:
        if i in aList:
            del aList[i]
    return aList

In [73]:
def score(guessList, realPlay):
    hit = 0
    for i in realPlay:
        if i in guessList:
            hit += 1
    return len(guessList), len(realPlay), hit

In [70]:
def getTop10(dicc, topN):
    ll = sorted(dicc.items(), key=lambda d: d[1], reverse=True)[:topN]
    # print len(ll)
    listToR = []
    for i in range(topN):
        # print i
        try:
            listToR.append(ll[i][0])
        except:
            break
    return listToR

In [15]:
def gene_video_label_dic(filename): #"video_label.csv"
    dic_to_r = {}
    with open(filename) as f:
        data = f.readlines()
    for i in data:
        label, video = i.strip('\n').split(',')
        dic_to_r[video] = label
    return dic_to_r

In [16]:
def newSimiDic_gaussian(user, sourceDic, video_label_dic, user_inter):
    dic_to_r = {}

    for video in sourceDic.keys():
        try:
            video_label = video_label_dic[video]

            # inter_dic = geneDic(user_inter_dic[user])

            try:
                inter = user_inter[video_label]

            except Exception as e:
                print (Exception, ':', e)
                inter = 1

            # dic_to_r[video] = float(inter)*float(sourceDic[video])
            dic_to_r[video] = float(sourceDic[video])

        except:
            dic_to_r[video] = sourceDic[video]

    return dic_to_r

In [76]:
def valid_gaussian(path, C_x, sigma, mostk, topN, models, cluster_video_label): # m=5 k=5 n=10
    # model = gensim.models.Word2Vec.load('G:/userBehavior/code/models/' + models)
    model = models
    data = gzip.open('D:/wenjun/testData/'+path).readlines()

    video_label_dic = gene_video_label_dic(cluster_video_label)
    user_inter_dic = pd.read_csv('D:/wenjun/xiaoming/C'+str(C_x)+'/user_inter_Guassian_sigma_'+str(sigma))
    user_inter_dic.rename(columns={'Unnamed: 0':'userid'}, inplace=True)

    hit, guessTotal, realTotal = 0, 0, 0
    count = 0
    top10 = []
    coverange = []
    #for line in data[100000:102500]:
    for line in data:
        try:
            user, videolist = line.decode('utf8').strip('\n').split('\t')
        except:
            continue
        # print user, videolist
        # print user
        vList, pList = [], []
        for x in videolist.split(' '):
            vList.append(x.split(':')[0])
            pList.append(x.split(':')[1])
        testList, validList = vList[0:5], vList[5:]
        # print testList, validList
        res = {}

        kl = [user]
        for video in testList:
            try:
                for similar_video in model.most_similar(video, topn=mostk):
                    if similar_video[0] in res:
                        res[similar_video[0]] = res[similar_video[0]] + similar_video[1]
                    else:
                        res[similar_video[0]] = similar_video[1]
            except Exception as e:
                #print Exception, ':', e
                continue


        user_inter = user_inter_dic[user_inter_dic['userid'] == user]

        pp = getTop10(newSimiDic_gaussian(user,
                                 delSame(res, testList), video_label_dic, user_inter), topN)



        if len(pp) is 0:
            kl.extend(['0'])
        else:
            kl.extend(pp)
        top10.extend(kl)
        guessLen, realLen, hitLen = score(pp, validList)

        guessTotal += guessLen
        realTotal += realLen
        hit += hitLen
        #
        # logging.info("count: "+str(count)+"/126211    Total Recomend video counts: "+str(guessTotal)+"    Real Play counts: "+
        #              str(realTotal)+"     Hits: "+str(hit))
        count += 1
        print("count: " + str(count) + "/126211    Total Recomend video counts: " + str(
        guessTotal) + "    Real Play counts: " + str(realTotal) + "     Hits: " + str(hit) \
           + '    Coverange video number: ' + str(len(set(top10))-2500))
    return "count: " + str(count) + "/126211    Total Recomend video counts: " + str(
        guessTotal) + "    Real Play counts: " + str(realTotal) + "     Hits: " + str(hit) \
           + '    Coverange video number: ' + str(len(set(top10))-2500)

In [ ]:
def valid_gaussian2(path, C_x, sigma, mostk, topN, models, cluster_video_label): # m=5 k=5 n=10
    # model = gensim.models.Word2Vec.load('G:/userBehavior/code/models/' + models)
    model = models
    
    f = open(seed_video_path, 'rb')
    seed_video = f.read()
    f.close()
    seed_video = eval(seed_video)
    
    f = open(test_seq_path, 'rb')
    test_seq = f.read()
    f.close()
    test_seq = eval(test_seq)

    video_label_dic = gene_video_label_dic(cluster_video_label)
    user_inter_dic = pd.read_csv('D:/wenjun/xiaoming/C'+str(C_x)+'/user_inter_Guassian_sigma_'+str(sigma))
    user_inter_dic.rename(columns={'Unnamed: 0':'userid'}, inplace=True)

    hit, guessTotal, realTotal = 0, 0, 0
    count = 0
    top10 = []
    coverange = []
    #for line in data[100000:102500]:
    testLlist = seed_video
    validLlist = test_seq
    for user in list(testList.keys):
        
        kl = [user]
        for video in testList[user]:
            try:
                for similar_video in model.most_similar(video, topn=mostk):
                    if similar_video[0] in res:
                        res[similar_video[0]] = res[similar_video[0]] + similar_video[1]
                    else:
                        res[similar_video[0]] = similar_video[1]
            except Exception as e:
                #print Exception, ':', e
                continue


        user_inter = user_inter_dic[user_inter_dic['userid'] == user]

        pp = getTop10(newSimiDic_gaussian(user,
                                 delSame(res, testList), video_label_dic, user_inter), topN)



        if len(pp) is 0:
            kl.extend(['0'])
        else:
            kl.extend(pp)
        top10.extend(kl)
        guessLen, realLen, hitLen = score(pp, validList)

        guessTotal += guessLen
        realTotal += realLen
        hit += hitLen
        #
        # logging.info("count: "+str(count)+"/126211    Total Recomend video counts: "+str(guessTotal)+"    Real Play counts: "+
        #              str(realTotal)+"     Hits: "+str(hit))
        count += 1
        print("count: " + str(count) + "/126211    Total Recomend video counts: " + str(
        guessTotal) + "    Real Play counts: " + str(realTotal) + "     Hits: " + str(hit) \
           + '    Coverange video number: ' + str(len(set(top10))-2500))
    return "count: " + str(count) + "/126211    Total Recomend video counts: " + str(
        guessTotal) + "    Real Play counts: " + str(realTotal) + "     Hits: " + str(hit) \
           + '    Coverange video number: ' + str(len(set(top10))-2500)

In [77]:
print( valid_gaussian2('part-r-00001.gz', 60, 70, 5, 10, word2vec_model, 'D:/wenjun/xiaoming/video_label_C' + str(60) + '.csv'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


count: 1/126211    Total Recomend video counts: 10    Real Play counts: 33     Hits: 2    Coverange video number: -2489
count: 2/126211    Total Recomend video counts: 20    Real Play counts: 53     Hits: 3    Coverange video number: -2478
count: 3/126211    Total Recomend video counts: 30    Real Play counts: 82     Hits: 11    Coverange video number: -2467
count: 4/126211    Total Recomend video counts: 40    Real Play counts: 98     Hits: 15    Coverange video number: -2456
count: 5/126211    Total Recomend video counts: 50    Real Play counts: 130     Hits: 19    Coverange video number: -2445
count: 6/126211    Total Recomend video counts: 60    Real Play counts: 158     Hits: 25    Coverange video number: -2434
count: 7/126211    Total Recomend video counts: 70    Real Play counts: 532     Hits: 44    Coverange video number: -2423
count: 8/126211    Total Recomend video counts: 80    Real Play counts: 575     Hits: 54    Coverange video number: -2413
count: 9/126211    Total Recom

KeyboardInterrupt: 